#### Problem Statement :
Building  an  Interpreter  in  Python  language  that  does  divide.  Work  with  tokens,  lexical analyzer, and expressions that divide integers from integers.

##### Steps involved in making an interpreter :
.1) Take input from the user in form of .txt file <br>
.2) Lexical analysis on the file to remove '\t' and whitespaces.<br>
.3) Store variables encountered in symbol table <br>
.4) Evaluate expressions by parsing through token streams how we do for infix expressions.<br>

#### .1) Code to accept .txt file as input
here 'lines' is a list containing all the lines

In [127]:
with open('input.txt') as f:
    lines = f.readlines()

print(lines)

['a = 23.3\n', 'b = a/5\n', 'c = d/2']


#### .2) Lexical analysis on the file to remove '\t' and whitespaces.
here we do lexical analysis on each line in 'lines'.

In [128]:
# We define our lexer function which tokenizes each line

def isDigit(a):
    """
    checks whether a character is digit or not.
    input : (char) accepts a character
    output : (bool) returns true if it is a digit 
    """
    if a>='0' and a<='9':
        return True
    return False

def isLetter(a):
    """
    checks whether a character is alphabet or not.
    input : (char) accepts a character
    output : (bool) returns true if it is a alphabet 
    """
    if (a>='a' and a<='z') or (a>='A' and a<='Z') or (a=='_'):
        return True
    return False


# We define some DFAs here

def DFA_identifier(s):
    """
    DFA to check if a string is an identifier or not
    input : (string) s
    output : (bool) returns true if s is an identifier.
    """
    if isLetter(s[0]):
        for i in range(1,len(s)):
            if isLetter(s[i]) or isDigit(s[i]):
                pass
            else:
                return False
        return True
    
    else:
        return False

def DFA_num(s):
    """
    DFA to check if a string is an number or not
    input : (string) s
    output : (bool) returns true if s is an number.
    """
    dec=False
    if isDigit(s[0]):
        for i in range(1,len(s)):
            if isDigit(s[i]) :
                pass
            elif s[i]=='.' and dec==False:
                dec=True
            else:
                return False
        return True
    else:
        return False

def DFA_equal(s):
    """
    DFA to check if a string is '='
    input : (string) s
    output : returns if string is equal to '='
    """
    return s=='='

def DFA_divide(s):
    """
    DFA to check if a string is '/'
    input : (string) s
    output : returns if string is equal to '/'
    """
    return s=='/'

def DFA_lparen(s):
    """
    DFA to check if a string is '('
    input : (string) s
    output : returns if string is equal to '('
    """
    return s=='('

def DFA_rparen(s):
    """
    DFA to check if a string is ')'
    input : (string) s
    output : returns if string is equal to ')'
    """
    return s==')'

# function checks for a character in a list of characters

def search(c):
    """
    checks if char c is present in list of characters
    input : (char) char to be checked
    output : (bool) returns true if present
    """
    delim = ['/','=','(',')',' ']
    for i in delim:
        if c==i:
            return True
    return False

In [129]:
def token(s):
    """
    returns the token type of the string.
    input : (string) accepts a string and returns token type.
    output : (string) returns the token type. 
    """
    
    if DFA_identifier(s):
        return 'IDENT'
    
    elif DFA_num(s):
        return 'NUM'
    
    elif DFA_equal(s):
        return 'EQ'
    
    elif DFA_divide(s):
        return 'DIV'
    
    elif DFA_lparen(s):
        return 'LPAREN'
    
    elif DFA_rparen(s):
        return 'RPAREN'
    
    return 'none'

def Lexer(s):
    """
    converts a line of code into stream of tokens.
    input : (string) accepts a line as string.
    output : (list of string) returns the stream of tokens. 
    """
    lexeme = []
    tokens = []
    
    ptr = 0
    
    for i in range(len(s)):
        if search(s[i]):
            if s[ptr:i] != '':
                lexeme.append(s[ptr:i])
            if s[i]!=' ':
                lexeme.append(s[i])
            ptr = i+1

    if s[ptr:] != '':
        lexeme.append(s[ptr:])
    
    for i in lexeme:
        tokens.append(token(i))
    
    return (tokens,lexeme)

In [130]:
# first we remove '\n' from end of each line if it is present

for i in range(len(lines)):
    if lines[i][-1] == '\n':
        lines[i] = lines[i][:-1]

# now we tokenize our lines of code

list_tokens = []

for i in range(len(lines)):
    list_tokens.append(Lexer(lines[i]))
        
print(list_tokens)

[(['IDENT', 'EQ', 'NUM'], ['a', '=', '23.3']), (['IDENT', 'EQ', 'IDENT', 'DIV', 'NUM'], ['b', '=', 'a', '/', '5']), (['IDENT', 'EQ', 'IDENT', 'DIV', 'NUM'], ['c', '=', 'd', '/', '2'])]


##### Compute line by line and store variables in symbol table

In [131]:
def comp_paren(lexeme, tokens, sym_tab):
    """
    computes expressions such as 1/((2/2)/3)
    Here, we will use the method we use to evaluate infix expressions.
    input : 
        (list of string) lexeme is the list of lexemes.
        (list of string) tokens is the list of tokens.
        (dict) sym_tab is the dictionary having the symbol table of our past variables.
    output : 
        (float) f is the float value we return after entire computation.
    """
    operand = []
    operator = []
    
    for i in range(len(tokens)):
        if tokens[i]=='NUM':
            operand.append(float(lexeme[i]))
        elif tokens[i]=='IDENT':
            if lexeme[i] in sym_tab:
                operand.append(sym_tab[lexeme[i]])
            else:
                return 'Error : Identifier '+str(lexeme[i])+' not declared.'
        elif tokens[i]=='LPAREN':
            operator.append('LPAREN')
        elif tokens[i]=='DIV':
            if len(operator)==0:
                operator.append('DIV')
            else:
                try:
                    oprtr_top = operator.pop()
                    if oprtr_top=='DIV':
                        oprnd_top_1 = operand.pop()
                        oprnd_top_2 = operand.pop()
                        if oprnd_top_1==0:
                            return 'Error : Cannot divide by zero.'
                        operand.append(oprnd_top_2/oprnd_top_1)
                        operator.append(tokens[i])
                    elif oprtr_top=='LPAREN':
                        operator.append(oprtr_top)
                        operator.append(tokens[i])
                    else:
                        return 'Error : Invalid expression.'
                except:
                    return 'Error : Invalid expression.'
        elif tokens[i]=='RPAREN':
            try:
                oprtr_top = operator.pop()
                while oprtr_top!='LPAREN':
                    oprnd_top_1 = operand.pop()
                    oprnd_top_2 = operand.pop()
                    if oprnd_top_1==0:
                        return 'Error : Cannot divide by zero.'
                    if oprtr_top=='DIV':
                        operand.append(oprnd_top_2/oprnd_top_1)
                    else:
                        return 'Error : Invalid operation.'
                    oprtr_top = operator.pop()
            except:
                return 'Error : Invalid expression.'
        else:
            return 'Error : '+str(lexeme[i])+' not a valid token.'
        
    while len(operator)!=0:
        try : 
            oprtr_top = operator.pop()
            oprnd_top_1 = operand.pop()
            oprnd_top_2 = operand.pop()
            if oprnd_top_1==0:
                return 'Error : Cannot divide by zero.'
            operand.append(oprnd_top_2/oprnd_top_1)
        except:
            return 'Error : Invalid expression.'
    
    if len(operand)==1:
        return operand.pop()
    return 'Error : Invalid expression.'

In [132]:
sym_tab = {}

f = open("output.txt", "w")

for i in range(len(list_tokens)):
    try:
        if list_tokens[i][0][1] == 'EQ':
            if list_tokens[i][0][0]=='IDENT':
                var = comp_paren(list_tokens[i][1][2:],list_tokens[i][0][2:],sym_tab)
                if type(var) == str:
                    print('Error in line ',i+1,': ',var)
                    f.write('Error in line '+str(i+1)+': '+str(var)+'\n')
                else:
                    sym_tab[list_tokens[i][1][0]] = var
                    print(var)
                    f.write(str(var)+'\n')
            else:
                print('Error in line '+str(i+1)+': Error :',list_tokens[i][1][0],' not a valid identifier.')
                f.write('Error in line '+str(i+1)+': Error :'+str(list_tokens[i][1][0])+' not a valid identifier.\n')
        else:
            var = comp_paren(list_tokens[i][1],list_tokens[i][0],sym_tab)
            if type(var) == str:
                print('Error in line ',i+1,' : ',var)
                f.write('Error in line '+str(i+1)+': '+str(var)+'\n')
            else:
                print(var)
                f.write(str(var)+'\n')
    except:
        print('Error : Invalid code.')
        f.write('Error in line '+str(i+1)+': Error : Invalid code.\n')
        
print(sym_tab)
f.close()

23.3
4.66
Error in line  3 :  Error : Identifier d not declared.
{'a': 23.3, 'b': 4.66}
